# INFO 2950 Project Phase 2

## I. Research Questions

write questions here


## II. Data Collecting and Cleaning

### A. Setup

Here is the setup part that imports everything needed throughout the rest of the notebook.  I copied the imports from homework 2 since they match what is needed here.

In [124]:
import sys
!conda install --yes --prefix {sys.prefix} requests
!conda install --yes --prefix {sys.prefix} bs4
import requests #package for http requests
import bs4 # package for html parsing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



### B. "Library" 

Although it's not an actual separate library, I ended up creating a lot of functions and some variables to help me clean up the data and this section is where they all live.  A description of what the functions do can be found in their respective docstrings and the motivations for using them are specified in the next section which is where all of the actual data cleaning magic takes place.

In [125]:
def web_to_soup(web_address, file_name):
    response = requests.get(web_address)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    file = file_name
    with open(file, mode='w', encoding='utf-8') as f:
        f.write(response.text)
    return soup

In [126]:
def extract_categories(soup):
    """ returns a list of categories from the left side of the samhsa data tables. """
    search = soup.find_all("th",{"class":["subhead2","subhead1"]}) 
    categories = []
    for tag in search:
        categories.append(tag.text)
    categories = np.array(categories)
    return categories

In [127]:
## pull out the data for each demographic category 
def pull_out_data(soup):
    search = soup.find_all("td") 
    data_entries = []
    for tag in search:
        data_entries.append(tag.text)
    data_entries = np.array(data_entries)
    return data_entries

In [321]:
def delete_blank(data_array):
    """delete blank rows"""
    to_delete = []
    for i in range(len(data_array)):
        if data_array[i]=='\xa0':
            to_delete.append(i)
    data = np.delete(data_array, to_delete)
    return data

In [335]:
def delete_footnotes_and_commas(dataframe):
    col_names = dataframe.columns.values.tolist()
    row_count = len(dataframe)
    for name in col_names:
        for i in range(row_count):
            if('a' in dataframe[name][i]):
                dataframe[name][i]=dataframe[name][i].replace('a', '')
            if(',' in dataframe[name][i]):
                dataframe[name][i]=dataframe[name][i].replace(',', '')
    return dataframe

In [328]:
def arrange_data(array, number_of_rows, number_of_columns):
    """ puts data in rows and columns """
    formatted_data=[]
    count=0
    for i in range(number_of_rows):
        formatted_data.append(array[count:count+number_of_columns:1])
        count+=number_of_columns
    return formatted_data

In [343]:
def to_numbers(dataframe):
    col_names = dataframe.columns.values.tolist()
    row_count = len(dataframe)
    for name in col_names:
        for i in range(row_count):
            dataframe[name][i]=pd.to_numeric(dataframe[name][i], 'coerce')
    return dataframe

In [609]:
years1 = ['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
years2 = ['2002', '2003', '2004', '2005', '2006', '2007','2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
ages1 = ['aged_12+_2018', 'aged_12+_2019','aged_12-17_2018', 'aged_12-17_2019','aged_18+_2018','aged_18+_2019', 'aged_18-25_2018','aged_18-25_2019', 'aged_26+_2018', 'aged_26+_2019']
ages2 = ['aged_18+_2018', 'aged_18+_2019','aged_18-25_2018', 'aged_18-25_2019','aged_26+_2018','aged_26+_2019', 'aged_26-49_2018','aged_26-49_2019', 'aged_50+_2018', 'aged_50+_2019']



### C. Data Cleaning Time!

First, I put all of the actual web scraping code in a cell of its own since it took a few seconds for the content of each page to be gathered and I wanted to make that a one-and-done task.  I used the function web_to_soup() that I wrote above to get the html content from the website specified in the argument web_address.  This function also dumps the html into a file whose name is given in the argument file_name.


In [617]:
soup1=web_to_soup('https://www.samhsa.gov/data/sites/default/files/reports/rpt29394/NSDUHDetailedTabs2019/NSDUHDetTabsSect10pe2019.htm', 'samhsa1.html')
soup2=web_to_soup('https://www.samhsa.gov/data/sites/default/files/reports/rpt29394/NSDUHDetailedTabs2019/NSDUHDetTabsSect11pe2019.htm', 'samhsa2.html')
soup3=web_to_soup('https://www.samhsa.gov/data/sites/default/files/reports/rpt29394/NSDUHDetailedTabs2019/NSDUHDetTabsSect1pe2019.htm', 'samhsa3.html')

Since our project incorporates a lot of different data from a lot of different data tables that don't intuitively merge well, I created a dictionary to store a description of the information provided by each DataFrame:

In [653]:
what_it_do = {'df1':'Mental illness numbers by demographic', 'df2':'Mental illness percent by demographic', 'df3': 'Locations where mental health services were received by age', 'df4': 'Locations where mental health services were received by age (percent)', 'df5': 'Illicit drug use percent by demographic', 'df6': 'Mental illness percent by demographic'}

In [630]:
## a bit of formatting/editing for the first table
demographics = extract_categories(soup1)
demographics1 = demographics[:14]
demographics1 = np.insert(demographics1,0,"total")

data1=pull_out_data(soup1)
data1=data1[:216]

data1=delete_blank(data1)

data1_formatted = arrange_data(data1, 15, 12)

df1 = pd.DataFrame(data1_formatted, demographics1, years)

df1 = delete_footnotes_and_commas(df1)

df1 = to_numbers(df1)

df1 = df1.transpose()

df1


,total,18-25,26 or Older,26-49,50 or Older,Male,Female,Not Hispanic or Latino,White,Black or African American,AIAN,NHOPI,Asian,Two or More Races,Hispanic or Latino
2008,39826,6099,33727,20697,13030,14778,25048,34971,28286,4314,163,NaN,1300,666,4855
2009,41195,6043,35152,21455,13698,15614,25581,36321,29799,4113,219,NaN,1287,757,4874
2010,41417,6151,35266,20648,14618,16461,24956,36580,29704,4520,187,NaN,1425,584,4837
2011,41381,6356,35025,19936,15089,15895,25486,36793,29552,4364,357,NaN,1473,847,4589
2012,43714,6774,36940,20777,16163,16856,26858,38049,30066,5044,366,NaN,1608,709,5665
2013,43846,6754,37092,21061,16031,16452,27394,37828,30222,4688,323,112,1489,995,6018
2014,43553,7014,36539,20007,16531,16371,27182,37795,30074,4603,271,204,1653,989,5758
2015,43421,7574,35847,20589,15257,16785,26636,37919,30387,4410,270,89,1620,1142,5502
2016,44652,7635,37016,20903,16114,17142,27510,38598,31266,4171,317,181,1602,1061,6053
2017,46632,8849,37783,22256,15527,18065,28567,40603,32183,4782,246,193,2000,1198,6028


In [631]:
demographics2 = demographics[14:28]
demographics2 = np.insert(demographics2,0,"total")
data2=pull_out_data(soup1)
data2 = data2[216:432]
data2 = delete_blank(data2)
data2_formatted = arrange_data(data2, 15, 12)
df2 = pd.DataFrame(data2_formatted, demographics2, years)
df2 = delete_footnotes_and_commas(df2)
df2 = df2.transpose()
df2 = to_numbers(df2)
df2

,total,18-25,26 or Older,26-49,50 or Older,Male,Female,Not Hispanic or Latino,White,Black or African American,AIAN,NHOPI,Asian,Two or More Races,Hispanic or Latino
2008,17.7,18.5,17.6,20.7,14.1,13.6,21.5,18.0,18.3,16.7,17.3,NaN,13.3,26.5,16.0
2009,18.1,18.0,18.2,21.6,14.5,14.2,21.8,18.5,19.2,15.6,20.0,NaN,12.8,31.1,15.7
2010,18.1,18.1,18.1,20.9,15.1,14.8,21.1,18.5,19.0,16.9,17.4,NaN,13.7,22.0,15.2
2011,17.8,18.5,17.7,20.3,15.0,14.2,21.1,18.5,19.0,16.3,27.7,NaN,13.1,25.8,13.5
2012,18.6,19.6,18.4,21.2,15.8,14.9,22.0,19.0,19.3,18.6,28.3,NaN,13.9,20.7,16.3
2013,18.5,19.4,18.3,21.5,15.3,14.4,22.3,18.7,19.3,16.9,26.0,14.4,12.3,28.1,16.9
2014,18.1,20.1,17.8,20.4,15.4,14.1,21.8,18.6,19.2,16.3,21.2,22.3,13.1,27.1,15.6
2015,17.9,21.7,17.2,20.9,14.0,14.3,21.2,18.5,19.3,15.4,21.2,14.8,12.0,29.5,14.5
2016,18.3,22.1,17.6,21.1,14.5,14.5,21.7,18.7,19.9,14.5,22.8,16.7,12.1,26.5,15.7
2017,18.9,25.8,17.8,22.2,13.8,15.1,22.3,19.6,20.4,16.2,18.9,19.4,14.5,28.6,15.2


In [620]:
locations1 = extract_categories(soup2)
locations1 = locations1[:17]
locations1 = np.insert(locations1,0,"Specialty Services")
locations1 = np.insert(locations1, 9, "Nonspecialty Services")
locations1 = np.append(locations1, "Specialty services in education, general medicine, or child welfare")
locations1 = np.delete(locations1,13)
locations1 = np.delete(locations1,15)
locations1 = np.delete(locations1,17)
data3=pull_out_data(soup3)
data3=data3[:360]
data3=delete_blank(data3)
data3_formatted = arrange_data(data3, 17, 18)
df3 = pd.DataFrame(data3_formatted, locations1, years2)
df3=delete_footnotes_and_commas(df3)
df3=to_numbers(df3)
df3=df3.transpose()




In [627]:
locations2=locations1
data4=pull_out_data(soup2)
data4=data4[360:720]
data4=delete_blank(data4)
data4=delete_footnotes(data4)
data4_formatted = arrange_data(data4, 17, 18)
df4 = pd.DataFrame(data4_formatted, locations1, years2)
df4 = df4.transpose()
df4 = to_numbers(df4)

,Specialty Services,Outpatient,"Private Therapist, Psychologist, Psychiatrist, Social Worker, or Counselor",Mental Health Clinic or Center,Partial Day Hospital or Day Treatment Program,"In-Home Therapist, Counselor, or Family Preservation Worker",Inpatient or Residential1,Hospital,Residential Treatment Center,Nonspecialty Services,"Education2,3","School Social Worker, School Psychologist, or School Counselor",Special School or Program within a Regular School for Students with Emotional or Behavioral Problems,Pediatrician or Other Family Doctor,Juvenile Justice,Child Welfare,Foster Care or Therapeutic Foster Care
2002,11.8,10.8,9.2,2.5,1.8,2.8,2.1,1.7,0.9,NaN,NaN,NaN,NaN,2.7,NaN,0.6,NaN
2003,12.4,11.3,9.5,2.6,1.7,2.6,2.2,1.9,0.9,NaN,NaN,NaN,NaN,2.9,NaN,0.7,NaN
2004,13.4,12.1,10.1,2.9,1.8,3.0,2.5,2.1,1.2,NaN,NaN,NaN,NaN,3.4,NaN,0.6,NaN
2005,13.4,12.1,10.2,2.6,1.8,2.9,2.5,2.1,0.9,NaN,NaN,NaN,NaN,3.2,NaN,0.6,NaN
2006,13.0,11.7,9.6,2.3,1.9,2.9,2.4,2.0,0.9,NaN,NaN,NaN,NaN,2.8,NaN,0.5,NaN
2007,12.4,11.2,9.5,2.3,1.7,2.8,2.3,2.0,0.8,NaN,NaN,NaN,NaN,2.8,NaN,0.5,NaN
2008,12.7,11.5,9.8,2.3,1.5,2.9,2.2,1.9,0.8,NaN,NaN,NaN,NaN,2.9,NaN,0.5,NaN
2009,12.0,10.9,9.4,2.2,1.4,2.7,2.1,1.8,0.9,14.2,12.1,9.4,4.0,2.5,0.4,0.4,5.0
2010,12.1,10.9,9.4,2.3,1.5,2.8,2.2,1.8,0.9,14.5,12.4,9.2,4.4,2.5,0.3,0.4,5.3
2011,12.6,11.5,9.8,2.2,1.6,3.1,2.1,1.8,1.0,14.2,11.9,8.9,4.3,2.5,0.4,0.6,5.4


In [629]:
demographics3 = extract_categories(soup3)
demographics3 = demographics3.tolist()
itemindex = demographics3.index('Male')
demographics3 = demographics3[itemindex:itemindex+18]
demographics3 = np.asarray(demographics3)
data5 = pull_out_data(soup3)
data5 = data5.tolist()
#data5.index('49.2a')
data5 = data5[5882:5882+230]
data5=delete_blank(data5)
data5_formatted = arrange_data(data5, 18, 10)
df5 = pd.DataFrame(data5_formatted, demographics3, ages1)
df5=delete_footnotes_and_commas(df5)
df5 = to_numbers(df5)
df5 = df5.transpose()
df5

,Male,Female,Not Hispanic or Latino,White,Black or African American,AIAN,NHOPI,Asian,Two or More Races,Hispanic or Latino,< High School,High School Graduate,Some College/Associate's Degree,College Graduate,Full-Time,Part-Time,Unemployed,Other1
aged_12+_2018,49.2,53.6,45.1,51.6,54.5,45.9,60.8,47.7,27.6,61.4,37.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aged_12+_2019,50.2,54.2,46.3,52.2,55.6,46.9,55.7,44.0,24.0,62.6,40.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aged_12-17_2018,23.9,23.8,24.0,23.5,23.4,26.2,28.4,NaN,13.7,28.7,25.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aged_12-17_2019,24.5,23.8,25.2,24.1,24.3,25.5,37.5,NaN,12.9,27.9,25.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aged_18+_2018,51.8,56.7,47.2,54.1,57.1,48.2,64.2,49.2,29.0,67.4,39.6,37.5,47.7,57.8,54.6,58.2,54.4,57.8
aged_18+_2019,52.7,57.4,48.3,54.8,58.2,49.3,58.0,45.5,24.9,70.0,42.3,40.8,49.2,58.9,53.9,58.5,55.8,57.6
aged_18-25_2018,55.6,56.2,54.9,56.6,59.6,53.1,56.0,NaN,36.8,68.7,52.0,47.7,53.1,58.9,59.1,60.4,55.1,58.4
aged_18-25_2019,55.8,55.9,55.7,57.1,60.4,52.8,64.0,NaN,35.3,64.0,51.6,50.0,52.1,58.3,62.7,62.9,54.4,53.1
aged_26+_2018,51.2,56.8,46.0,53.8,56.8,47.2,65.6,NaN,27.5,67.1,36.8,35.7,46.6,57.6,54.3,58.0,54.2,57.5
aged_26+_2019,52.2,57.7,47.2,54.4,57.9,48.6,56.8,46.4,23.2,71.6,40.2,39.1,48.6,59.0,53.3,57.9,56.3,59.8


In [632]:
## data is from the following set of tables: https://www.samhsa.gov/data/sites/default/files/reports/rpt29394/NSDUHDetailedTabs2019/NSDUHDetTabsSect8pe2019.htm

data6=[]
data6.append([19.1,20.6,6.3,29.4,18.0,19.2,22.5,25.0,14.0,14.1])
data6.append([15.2,16.3,20.6,22.8,14.3,15.3,18.1,19.8,10.8,11.2])
data6.append([22.8,24.5,32.0,36.1,21.4,22.8,26.8,30.0,16.9,16.7])
data6.append([19.6,21.1,27.7,31.0,18.4,19.7,23.7,26.6,14.2,14.2])
data6.append([20.4,22.2,29.9,33.4,19.2,20.7,26.1,29.2,14.3,14.8])
data6.append([16.2,17.3,20.4,23.8,15.4,16.1,16.8,20.3,14.0,11.7])
data6.append([22.1,18.7,22.6,23.6,22.0,17.7,26.1,22.1,np.NaN,np.NaN])
data6.append([21.1,16.6,np.NaN,np.NaN,np.NaN,15.1,np.NaN,20.2,np.NaN,np.NaN])
data6.append([14.7,14.4,23.7,24.6,13.0,12.7,16.0,15.7,7.8,8.6])
data6.append([26.8,31.7,37.0,39.3,24.6,29.6,29.2,36.3,21.4,20.8])
data6.append([16.9,18.0,21.3,24.0,15.9,16.7,17.6,18.5,13.0,13.7])
data6.append([17.6,19.2,25.0,28.2,16.7,18.0,19.6,22.3,11.7,10.7])
data6.append([22.0,25.0,29.5,34.6,19.3,21.7,26.6,30.8,13.0,14.4])
data6.append([27.6,27.7,27.9,27.2,27.4,27.9,29.2,30.7,23.7,22.9])
data6.append([19.3,20.1,24.5,27.1,18.7,19.4,29.4,30.4,15.5,16.1])

demographics4 = demographics3
demographics4[10]='Full Time'
demographics4[11]='Part Time'
demographics4[12]='Unemployed'
demographics4[13]='Other'
demographics4=demographics4[:14]
demographics4=np.asarray(demographics4)
demographics4=np.insert(demographics4,0,"total")

df6 = pd.DataFrame(data6, demographics4, ages2)

df6 = df6.transpose()
df6


,total,Male,Female,Not Hispanic or Latino,White,Black or African American,AIAN,NHOPI,Asian,Two or More Races,Hispanic or Latino,Full Time,Part Time,Unemployed,Other
aged_18+_2018,19.1,15.2,22.8,19.6,20.4,16.2,22.1,21.1,14.7,26.8,16.9,17.6,22.0,27.6,19.3
aged_18+_2019,20.6,16.3,24.5,21.1,22.2,17.3,18.7,16.6,14.4,31.7,18.0,19.2,25.0,27.7,20.1
aged_18-25_2018,6.3,20.6,32.0,27.7,29.9,20.4,22.6,NaN,23.7,37.0,21.3,25.0,29.5,27.9,24.5
aged_18-25_2019,29.4,22.8,36.1,31.0,33.4,23.8,23.6,NaN,24.6,39.3,24.0,28.2,34.6,27.2,27.1
aged_26+_2018,18.0,14.3,21.4,18.4,19.2,15.4,22.0,NaN,13.0,24.6,15.9,16.7,19.3,27.4,18.7
aged_26+_2019,19.2,15.3,22.8,19.7,20.7,16.1,17.7,15.1,12.7,29.6,16.7,18.0,21.7,27.9,19.4
aged_26-49_2018,22.5,18.1,26.8,23.7,26.1,16.8,26.1,NaN,16.0,29.2,17.6,19.6,26.6,29.2,29.4
aged_26-49_2019,25.0,19.8,30.0,26.6,29.2,20.3,22.1,20.2,15.7,36.3,18.5,22.3,30.8,30.7,30.4
aged_50+_2018,14.0,10.8,16.9,14.2,14.3,14.0,NaN,NaN,7.8,21.4,13.0,11.7,13.0,23.7,15.5
aged_50+_2019,14.1,11.2,16.7,14.2,14.8,11.7,NaN,NaN,8.6,20.8,13.7,10.7,14.4,22.9,16.1
